<a href="https://colab.research.google.com/github/Mariyselita/CIP/blob/main/Algoritmos_de_clasificaci%C3%B3n_TAREA_9_CIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
filepath = '/content/drive/My Drive/CIP/Datos/'

In [10]:
import pandas as pd
from sklearn.utils import resample

# Cargar los datasets
ciclones_df = pd.read_csv(filepath + 'Ciclones.csv')
clima_extremo_df = pd.read_csv(filepath + 'Clima_Extremo.csv')
incendios_forestales_df = pd.read_csv(filepath + 'Incendios_Forestales.csv')
sequias_df = pd.read_csv(filepath + 'Sequías.csv')
sismos_df = pd.read_csv(filepath + 'Sismos.csv')

# Función para balancear los datasets
def balance_dataset(df, class_name):
    df['Clase'] = class_name
    majority_count = df['Clase'].value_counts().max()

    balanced_df = pd.DataFrame()

    for label in df['Clase'].unique():
        df_label = df[df['Clase'] == label]
        if len(df_label) < majority_count:
            df_label = resample(df_label, replace=True, n_samples=majority_count, random_state=42)
        balanced_df = pd.concat([balanced_df, df_label])

    return balanced_df

# Agregar columna 'Clase' y balancear los datasets
datasets_balanced = {
    "Ciclones": balance_dataset(ciclones_df, 'Ciclones'),
    "Clima Extremo": balance_dataset(clima_extremo_df, 'Clima Extremo'),
    "Incendios Forestales": balance_dataset(incendios_forestales_df, 'Incendios Forestales'),
    "Sequías": balance_dataset(sequias_df, 'Sequías'),
    "Sismos": sismos_df[sismos_df['Magnitud'] >= 3.5].assign(Clase='Sismos')
}

# Determinar el tamaño del dataset más pequeño después del filtrado
min_size = min([len(df) for df in datasets_balanced.values()])

# Balancear todos los datasets al tamaño mínimo
balanced_datasets_final = {
    name: resample(df, replace=False, n_samples=min_size, random_state=42) for name, df in datasets_balanced.items()
}

# Guardar los datasets balanceados en archivos CSV
for name, df in balanced_datasets_final.items():
    df.to_csv(filepath + f'balanced_{name}.csv', index=False)

# Mostrar la distribución de clases en los datasets balanceados finales
for name, df in balanced_datasets_final.items():
    print(f"Dataset: {name}")
    print(df['Clase'].value_counts())
    print("\n")


Dataset: Ciclones
Clase
Ciclones    291
Name: count, dtype: int64


Dataset: Clima Extremo
Clase
Clima Extremo    291
Name: count, dtype: int64


Dataset: Incendios Forestales
Clase
Incendios Forestales    291
Name: count, dtype: int64


Dataset: Sequías
Clase
Sequías    291
Name: count, dtype: int64


Dataset: Sismos
Clase
Sismos    291
Name: count, dtype: int64




In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [18]:
df_ciclones = pd.read_csv(filepath + 'balanced_Ciclones.csv')
df_clima_extremo = pd.read_csv(filepath + 'balanced_Clima Extremo.csv')
df_incendios_forestales = pd.read_csv(filepath + 'balanced_Incendios Forestales.csv')
df_sequías = pd.read_csv(filepath + 'balanced_Sequías.csv')
df_sismos = pd.read_csv(filepath + 'balanced_Sismos.csv')

# Añadir una columna de clase específica para cada dataset
df_ciclones['Clase'] = 'Ciclones'
df_clima_extremo['Clase'] = 'Clima Extremo'
df_incendios_forestales['Clase'] = 'Incendios Forestales'
df_sequías['Clase'] = 'Sequías'
df_sismos['Clase'] = 'Sismos'

# Eliminar la columna 'Fecha' si existe
for df in [df_ciclones, df_clima_extremo, df_incendios_forestales, df_sequías, df_sismos]:
    if 'Fecha' in df.columns:
        df.drop(columns=['Fecha'], inplace=True)

# Combinar los datasets en un solo dataframe
df = pd.concat([df_ciclones, df_clima_extremo, df_incendios_forestales, df_sequías, df_sismos], ignore_index=True)

# Eliminar filas con valores NaN en las características que vamos a usar
features = ['Ubicación', 'Magnitud', 'Duración', 'Áreas Afectadas']
features = [f for f in features if f in df.columns]
df = df.dropna(subset=features)

# Verificar la distribución de la clase
print(df['Clase'].value_counts())

# Seleccionar características y etiquetas
X = df[features]
y = df['Clase']  # Asegurarse de que la columna de etiquetas sea 'Clase'

# Convertir características categóricas a numéricas
X = pd.get_dummies(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Realizar predicciones
y_pred = clf.predict(X_test)

# Evaluar el modelo
print(classification_report(y_test, y_pred))

Clase
Sismos    291
Name: count, dtype: int64
              precision    recall  f1-score   support

      Sismos       1.00      1.00      1.00        59

    accuracy                           1.00        59
   macro avg       1.00      1.00      1.00        59
weighted avg       1.00      1.00      1.00        59

